In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from SSLRS.core import *
from fastai.vision.all import *
from fastai.distributed import *
import timm
import wandb

In [ ]:
df=pd.read_csv('./data/file.csv')

In [ ]:
newdf=pd.concat([df[df['Isval']==0].sample(frac=0.05,random_state=10),df[df['Isval']==1]])

In [ ]:
newdf.tail()

,fname,Isval,label
393413,./data/BigEarthNet-v1.0/S2B_MSIL2A_20170924T93021_75_6/S2B_MSIL2A_20170924T93021_75_6.tif,1,9 10
393414,./data/BigEarthNet-v1.0/S2B_MSIL2A_20180522T093029_72_7/S2B_MSIL2A_20180522T093029_72_7.tif,1,6 8 9 10 17
393415,./data/BigEarthNet-v1.0/S2B_MSIL2A_20180522T093029_75_6/S2B_MSIL2A_20180522T093029_75_6.tif,1,9 10
393416,./data/BigEarthNet-v1.0/S2B_MSIL2A_20170812T092029_72_7/S2B_MSIL2A_20170812T092029_72_7.tif,1,6 8 9 10 17
393417,./data/BigEarthNet-v1.0/S2B_MSIL2A_20170812T092029_75_6/S2B_MSIL2A_20170812T092029_75_6.tif,1,9 10


In [ ]:
db = DataBlock(blocks=(TransformBlock(type_tfms=partial(MSTensorImage.create)), MultiCategoryBlock),
                   splitter=ColSplitter('Isval'),
                   get_x=ColReader('fname'),
                   get_y=ColReader('label', label_delim=' '),
                   item_tfms=[aug,aug2]
              )
dls = db.dataloaders(source=newdf, bs=64, num_workers=0)

In [ ]:
# model=timm.create_model('convmixer_768_32',num_classes=19, in_chans=10)
model=timm.create_model('xcit_small_12_p8_224',num_classes=0, in_chans=10)

In [ ]:
state_dict=torch.load('./models/checkpoint0020.pth')['teacher']

In [ ]:
state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}
state_dict = {k.replace("backbone.", ""): v for k, v in state_dict.items()}

In [ ]:
model.load_state_dict(state_dict,strict=False)

_IncompatibleKeys(missing_keys=[], unexpected_keys=['head.mlp.0.weight', 'head.mlp.0.bias', 'head.mlp.2.weight', 'head.mlp.2.bias', 'head.mlp.4.weight', 'head.mlp.4.bias', 'head.last_layer.weight_g', 'head.last_layer.weight_v'])

In [ ]:
head=nn.Linear(384, 19)
init_linear(head)

In [ ]:
newmodel=nn.Sequential(model,head)

In [ ]:
# model=xresnet18(c_in=10, n_out=19)
# init_cnn(model)

In [ ]:
def model_split(model):
    groups = L([model[0], model[1]])
    return groups.map(params)

In [ ]:
learn = Learner(dls, newmodel ,splitter=model_split,metrics=partial(accuracy_multi, thresh=0.5)).to_fp16()

In [ ]:
learn.freeze()
# learn.lr_find()

In [ ]:
# wandb.init(project="SSLRS", entity="robeson")

In [ ]:
from fastai.callback.wandb import *


In [ ]:
# learn.fit_flat_cos(50,1e-3,cbs=WandbCallback(log_preds=False))
learn.fit_flat_cos(10,1e-3)

epoch,train_loss,valid_loss,accuracy_multi,time
0,0.225716,0.208975,0.910292,20:05
1,0.206658,0.198800,0.913705,09:35
2,0.197776,0.194204,0.915797,09:38


KeyboardInterrupt: 

In [ ]:
with learn.distrib_ctx(): learn.fit_flat_cos(2, 1e-3,cbs=[CSVLogger(fname='supres18.csv'),SaveModelCallback(monitor='accuracy_multi',fname='supres18')])